In [25]:
import requests
import pandas as pd
import numpy as np
from IPython.display import JSON
import os
api_key = os.environ["FOURSQUARE_API_KEY"]
YELP_api_key = os.environ["YELP_API_KEY"]

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [70]:
# Define a function to take lat and long coordinates and return POI data near it
def get_POI(latitude,longitude):
    
    url = "https://api.foursquare.com/v3/places/search?fields=name,geocodes,categories,rating,popularity,price"
    
    headers = {"Accept": "application/json"}
    headers['Authorization'] = api_key
    
    location = [latitude,longitude]
    location_str = [str(x) for x in location]
    location_join = ','.join(location_str)
    
    params_dict = {
                   "ll" : location_join,
                   "radius" : "1000",
                   
    }
    
    response = requests.get(url, params_dict, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f'Error {response.status_code}')
    

In [71]:
# Go through the lat and long coordinates of the bikestations and apply the get_POI funciton to it
df1 = pd.read_csv('bike_locations.csv')
df2 = df1.apply(
        lambda row: get_POI(row['latitude'], row['longitude']),
        axis=1)


In [91]:
#df3 = list(df2)

In [90]:
#df2[0]

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [74]:
fs_api = []
for data in df2:
    for val in data['results']:
        if 'rating' in val:
            fs_api.append([val['name'],val['categories'][0]['name'],val['geocodes']['main']['latitude'], val['geocodes']['main']['longitude'], val['rating'], val['popularity']])
   

Put your parsed results into a DataFrame

In [75]:
df_fs_api = pd.DataFrame(fs_api, columns = ['name', 'category', 'latitude', 'longitude', 'rating', 'popularity'])
df_fs_api

,name,category,latitude,longitude,rating,popularity
0,Vogue Fitness - Crossfit Yas,Sports and Recreation,24.466085,54.609540,8.2,0.990650
1,Stars N Bars,Bar,24.466085,54.609540,7.8,0.985040
2,Cipriani Yas Island,Italian Restaurant,24.467130,54.609222,8.9,0.977067
3,Diablito,Italian Restaurant,24.465609,54.609099,7.1,0.896271
4,Iris Yas Island,Lounge,24.467122,54.609105,8.7,0.958685
...,...,...,...,...,...,...
295,Aisha Lobby Lounge,Café,25.663201,55.749076,7.5,0.943775
296,Toast'n Burger,Burger Joint,25.664295,55.748702,6.9,0.938938
297,Mojito Bar,Cocktail Bar,25.663866,55.749080,7.3,0.204071
298,Al Hamra Cellar,Food and Beverage Retail,25.693119,55.796613,9.1,0.985335


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [63]:
def get_POI_yelp(latitude,longitude):
    
    url = "https://api.yelp.com/v3/businesses/search"
    
    headers = {"accept": "application/json",
              "Authorization": 'bearer %s' % YELP_api_key}

    
    
    params_dict = {
                   "term" : "food",
                   "latitude" : latitude,
                   "longitude" : longitude,
                   "radius" : 1000,
    }
    
    response = requests.get(url, params=params_dict, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f'Error {response.status_code}')
    

In [64]:
df_y = pd.read_csv('bike_locations.csv')
df_y1 = df_y.apply(
        lambda row: get_POI_yelp(row['latitude'], row['longitude']),
        axis=1)



In [65]:
df_y1[10]
# I don't believe yelp is collecting data in Abu Dhabi


{'businesses': [],
 'total': 0,
 'region': {'center': {'longitude': 54.619692999999984,
   'latitude': 24.441122}}}

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [66]:
# checking if the Yelp call had any data in them
for row in df_y1:
    if row['total'] > 0:
        print(row)
    else:
        print(f'No businesses found')
        
# I would have switched to another city if I had more time

No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found
No businesses found


Put your parsed results into a DataFrame

"NA"

In [68]:
"NA"

'NA'

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

In my case Yelp was not collecting ay data about businesses in Abu Dhabi, therefore the obvious choice is foursquare. However after looking at yelp documentation Yelp provides more detailed information about businesses with simpler queries for the cities they DO collect data from.

Get the top 10 restaurants according to their rating

In [85]:
#df_fs_api

In [84]:
df_fs_api[df_fs_api['category'].str.contains('Restaurant')].sort_values('rating', ascending=False).head(10)

,name,category,latitude,longitude,rating,popularity
237,Catch,Seafood Restaurant,24.465627,54.327545,9.0,0.931623
2,Cipriani Yas Island,Italian Restaurant,24.467130,54.609222,8.9,0.977067
181,Qasmieh Al Fawal,Middle Eastern Restaurant,24.471854,54.347455,8.7,0.980171
123,Qasmieh Al Fawal,Middle Eastern Restaurant,24.471854,54.347455,8.7,0.980171
119,Sushi Story,Japanese Restaurant,24.481588,54.351395,8.7,0.961472
138,Napo Ristorante,Italian Restaurant,24.452015,54.334343,8.5,0.988890
80,Fouquet's Abu Dhabi,French Restaurant,24.534417,54.398200,8.4,0.961944
206,Shish Shawerma Corniche,Middle Eastern Restaurant,24.470986,54.340719,8.4,0.974809
166,Fouquet's Abu Dhabi,French Restaurant,24.534417,54.398200,8.4,0.961944
133,Sushi Central,Japanese Restaurant,24.507550,54.376616,8.3,0.848287


In [87]:
df_fs_api.to_csv('fs_data.csv', columns = ['name', 'category', 'latitude', 'longitude', 'rating', 'popularity'], index=False)